In [1]:
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from chardet import detect
from chardet.universaldetector import UniversalDetector as udet
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from tqdm import tqdm
from cimaq_utils import flatten
from cimaq_utils import loadimages
from cimaq_utils import get_encoding


In [25]:
def loadpaths():
#     cimaq_dir='~/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA'
    cimaq_dir='~/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA'
    cimaq_dir = xpu(cimaq_dir)
    sheetpaths = df(((sub, [itm for itm in sorted(loadimages(join(cimaq_dir,
                                              'extracted_eprimes2021', sub)))
                            if 'CIMAQ' in bname(itm) and '._' not in itm])
                  for sub in ls(join(cimaq_dir, 'extracted_eprimes2021')))).set_index(0)
    prefixes = sorted(list(dict.fromkeys([bname(str(item)).split("CIMAQ", 1)[0] 
                    for item in flatten(sheetpaths.values.tolist())
                    if 'CIMAQ' in bname(item) and '._' not in item])))
    sheetpaths[prefixes] = [val[0] for val in sheetpaths.values]
    sheetpaths = sheetpaths[prefixes]
    
    return sheetpaths, prefixes
frame, prefixes = loadpaths()
display(frame, prefixes)

Onset-Event-Encoding_  \
0                                                                
sub-3025432  /home/francois/../../media/francois/seagate_8t...   
sub-3123186  /home/francois/../../media/francois/seagate_8t...   
sub-3163875  /home/francois/../../media/francois/seagate_8t...   
sub-3291977  /home/francois/../../media/francois/seagate_8t...   
sub-3303819  /home/francois/../../media/francois/seagate_8t...   
...                                                        ...   
sub-9003010  /home/francois/../../media/francois/seagate_8t...   
sub-9249304  /home/francois/../../media/francois/seagate_8t...   
sub-9296157  /home/francois/../../media/francois/seagate_8t...   
sub-9494076  /home/francois/../../media/francois/seagate_8t...   
sub-9929164  /home/francois/../../media/francois/seagate_8t...   

                                    Output-Responses-Encoding_  \
0                                                                
sub-3025432  /home/francois/../../media/francois/seagate_8t...   
sub-3123186  /home/francois/../../media/francois/seagate_8t...   
sub-3163875  /home/francois/../../media/francois/seagate_8t...   
sub-3291977  /home/francois/../../media/francois/seagate_8t...   
sub-3303819  /home/francois/../../media/francois/seagate_8t...   
...                                                        ...   
sub-9003010  /home/francois/../../media/francois/seagate_8t...   
sub-9249304  /home/francois/../../media/francois/seagate_8t...   
sub-9296157  /home/francois/../../media/francois/seagate_8t...   
sub-9494076  /home/francois/../../media/francois/seagate_8t...   
sub-9929164  /home/francois/../../media/francois/seagate_8t...   

                                             Output_Retrieval_  
0                                                               
sub-3025432  /home/francois/../../media/francois/seagate_8t...  
sub-3123186  /home/francois/../../media/francois/seagate_8t...  
sub-3163875  /home/francois/../../media/francois/seagate_8t...  
sub-3291977  /home/francois/../../media/francois/seagate_8t...  
sub-3303819  /home/francois/../../media/francois/seagate_8t...  
...                                                        ...  
sub-9003010  /home/francois/../../media/francois/seagate_8t...  
sub-9249304  /home/francois/../../media/francois/seagate_8t...  
sub-9296157  /home/francois/../../media/francois/seagate_8t...  
sub-9494076  /home/francois/../../media/francois/seagate_8t...  
sub-9929164  /home/francois/../../media/francois/seagate_8t...  

[103 rows x 3 columns]

['Onset-Event-Encoding_', 'Output-Responses-Encoding_', 'Output_Retrieval_']

In [26]:
def getid(sheet):
    nameparts = [str(item) for item
                 in os.path.splitext(bname(sheet))[0].split("_")]
    subid = "sub-"+str(nameparts[nameparts.index(\
                 (item for item in nameparts
                   if item.isnumeric()).__next__())])
    return subid

def makename(sheet, prefixes):
    if prefixes[0] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_onset"+".tsv")
    if prefixes[1] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_output"+".tsv")
    if prefixes[2] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-Retrieval"+".tsv")
    else:
        print("python no compute: " + sheet)


In [27]:
def get_enc_onsets():
    # Names from M. St-Laurent (2019)
    EncOnsetCols = ["TrialNum", "Condition", "TrialNum_perCondi",
                    "ImageID", "Trial_part", "onsetSec", "durationSec"]
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,0]
    s_ids, yallofems = [], []
    for sheet in sheets:
        newsheet = pd.read_fwf(sheet, encoding=get_encoding(sheet)[1]['encoding'],
                               header=None, sep='\t',
                               names=EncOnsetCols).fillna(False).iloc[6:]

        stimids = newsheet[["ImageID",
                           "TrialNum_perCondi"]].drop_duplicates(subset=["ImageID",
                              "TrialNum_perCondi"]).reset_index(drop=True)
        s_ids.append((bname(sheet[0]), stimids))
        newsheets = newsheet.drop(['TrialNum_perCondi', 'Condition'], axis=1)
        tempsheet = newsheet[['TrialNum', 'Trial_part',
                              'onsetSec', 'durationSec']]

        # Extract & concatenate relevant info
        fixsheet = df([sheet[1] for sheet in tempsheet.iterrows()
                       if sheet[1]['Trial_part'] == 'Fixation'])
        tmng = tempsheet.loc[[sheet[0]
                                for sheet in tempsheet.iterrows()
                                if sheet[0] not in fixsheet.index]]
        fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
                                            "durationSec": "fixDurSec"})
        fixsheet = fixsheet.transpose().iloc[-2:].transpose().reset_index(drop=True)
        tmng = tmng.rename(columns={"onsetSec": "stimOnsetSec",
                                        "durationSec": "stimDurSec"})
        tmng = tmng.transpose().iloc[-2:].transpose().reset_index(drop=True)
        allofem = pd.concat([tmng, fixsheet, stimids], axis=1, sort=False)
        allofem = allofem.rename(columns={"TrialNum_perCondi": "tNumXcond"})
        enconsTuple = (makename(sheet, prefixes), allofem)
        yallofems.append(enconsTuple)

    return sorted(yallofems)

######## ######## ######## ######## ######## ######## ######## ######## #######
# yallofems = get_enc_onsets()
# display(yallofems)

In [28]:
def get_enc_outputs():
#     sheets = frame.iloc[:,1]
    encsheets = []
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,1]
    for sheet in sheets:
        encsheet = pd.read_csv(sheet, encoding=get_encoding(sheet)[1]['encoding'],
                               header=0,
                               sep='\t').fillna(False).rename(
                       columns={"TrialNumber": "TrialNum",
                                "Category": "Condition"}).iloc[3:]
        encsheet = encsheet.set_index("TrialNum")
        # Drop duplicate columns with enc_onset's
        encsheet = encsheet.drop(["TrialCode", "OldNumber",
                                  "Stim_RESP"], axis=1)
        encsheet["Condition"] = encsheet["Condition"].astype('str')
        # Convert ms to s (as are all other time measures)
        encsheet["Stim_RT"] = [val/1000 for val in encsheet["Stim_RT"]]
        encsheet = encsheet.drop(["Stim_ACC"], axis=1)
        encsheet = encsheet.reset_index(drop=True)
        encoutTuple = (makename(sheet, prefixes), encsheet)
        encsheets.append(encoutTuple)
    return sorted(encsheets)
#     return sheets
# encsheets = get_enc_onsets()

In [30]:
def get_ret_outputs():
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,2]
    retsheets = []
    for sheet in sheets:
        # Removing last column as an Eprime error occured (St-Laurent, 2019)
        retsheet = pd.read_csv(sheet, encoding=get_encoding(sheet)[1]['encoding'], header=0,
                               sep='\t').fillna(False).iloc[:, :-1]
        retsheet["Recognition_RT"] = [float(val)/1000 for val
                                      in retsheet["Recognition_RT"]]
        retTuple = (makename(sheet, prefixes), retsheet)
        retsheets.append(retTuple)
    return sorted(retsheets)
    return sheets
# retsheets = get_ret_outputs()

In [80]:
def clean_all_events(cimaq_dir='~/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA'):
    cimaq_dir = xpu(cimaq_dir)
    frame, prefixes = loadpaths()

    enconsets = get_enc_onsets()
    encoutputs = get_enc_outputs()
    retsheets = get_ret_outputs()
    sheetspersub = tuple(zip(frame.index, enconsets, encoutputs, retsheets))
    fullencsheets = [(join(cimaq_dir, 'cimaq_enc_ret2021',
                           frame.index[item[0]]+"_task-Memory_events.tsv"),
                      pd.concat([item[1][1][1], item[1][2][1]], axis=1,
                      sort=False).reset_index(\
                          drop=True))
                     for item in enumerate(sheetspersub)]
    frame['Encoding'] = [join(cimaq_dir, 'cimaq_enc_ret2021',
                              row[0], row[0]+"_task-Memory_events.tsv")
                        for row in frame.iterrows()]    
    frame['Retrieval'] = [join(cimaq_dir, 'cimaq_enc_ret2021',
                              row[0], row[0]+"_task-Retrieval_behavioural.tsv")
                          for row in frame.iterrows()]
    [os.makedirs(dname(ndir), exist_ok=True) for ndir in frame.Encoding]
    retsvr = tuple(zip(frame.Retrieval, [itm[1] for itm in retsheets]))
    encsvr = tuple(zip(frame.Encoding, [itm[1] for itm in fullencsheets]))
    [itm[1].to_csv(itm[0], sep='\t') for itm in retsvr]
    [itm[1].to_csv(itm[0], sep='\t') for itm in encsvr]
    with open(join(cimaq_dir, 'cimaq_clean_unmerged.json'),
              'w', encoding='utf-8') as f:
        json.dump(np.array2string(np.array(sheetspersub, dtype='object')),
                  f, ensure_ascii=False, indent=0)
        f.close()
    frame.to_csv(join(cimaq_dir, 'cimaq_paths.tsv'), sep='\t')
clean_all_events()

# def main():
#     clean_all_events()

# if __name__ == '__main__':
#     main()

In [77]:
# display(fullencsheets[0], retsheets[0])
display(frame.Encoding[22], frame.Retrieval[22])

'/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/cimaq_enc_ret2021/sub-4206804/sub-4206804_task-Memory_events.tsv'

'/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/cimaq_enc_ret2021/sub-4206804/sub-4206804_task-Retrieval_behavioural.tsv'

In [61]:
display(sheetspersub)

'"[[\'sub-3025432\'\\n  (\'/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/extracted_eprimes2021/sub-3025432/sub-3025432_task-memory_events_onset.tsv\',     stimOnsetSec stimDurSec fixOnsetSec fixDurSec ImageID tNumXcond\\n0         11.542        3.0      14.551       5.5   False     CTL01\\n1         20.051        3.0      23.060       0.5   Old60     Enc01\\n2         23.561        3.0      26.570       1.0   Old01     Enc02\\n3         27.557        3.0      30.566       1.0   Old24     Enc03\\n4         31.569        3.0      34.578       1.5   Old68     Enc04\\n..           ...        ...         ...       ...     ...       ...\\n112      694.410        3.0     697.419       6.0   Old45     Enc75\\n113      703.420        3.0     706.429      10.0   False     CTL39\\n114      716.927        3.0     719.936       5.0   Old18     Enc76\\n115      724.934        3.0     727.943       1.0   Old33     Enc77\\n116      728.929        3.0     731.938      18.

In [52]:
display(retsheets[0], fullencsheets[0])

('/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/extracted_eprimes2021/sub-3025432/sub-3025432_task-Retrieval.tsv',
     category                          Stim OldNumber  Recognition_ACC  \
 0        OLD            animal_penguin.bmp     Old10                1   
 1        OLD      sporting_bicycle_old.bmp     Old55                0   
 2        OLD           fruit_lemon_new.bmp     Old69                1   
 3        New            vegie_broccoli.bmp     New36                1   
 4        OLD          animal_zebra_old.bmp     Old13                1   
 ..       ...                           ...       ...              ...   
 112      New       kitchen_spatula_old.bmp     New37                0   
 113      OLD        musical_violin_old.bmp     Old51                1   
 114      OLD              vegie_carrot.bmp     Old78                1   
 115      OLD  sporting_karate clothing.bmp     Old60                1   
 116      OLD        fruit_green grapes.

('/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/cimaq_enc_ret2021/sub-3025432',
     stimOnsetSec stimDurSec fixOnsetSec fixDurSec ImageID tNumXcond Condition  \
 0         11.542        3.0      14.551       5.5   False     CTL01       CTL   
 1         20.051        3.0      23.060       0.5   Old60     Enc01       Enc   
 2         23.561        3.0      26.570       1.0   Old01     Enc02       Enc   
 3         27.557        3.0      30.566       1.0   Old24     Enc03       Enc   
 4         31.569        3.0      34.578       1.5   Old68     Enc04       Enc   
 ..           ...        ...         ...       ...     ...       ...       ...   
 112      694.410        3.0     697.419       6.0   Old45     Enc75       Enc   
 113      703.420        3.0     706.429      10.0   False     CTL39       CTL   
 114      716.927        3.0     719.936       5.0   Old18     Enc76       Enc   
 115      724.934        3.0     727.943       1.0   Old33     Enc77 

In [ ]:
#     oldevnts = flatten([[sheet for sheet in loadimages(join(indir, row[0]))
#                          if "task-Memory_events.tsv" in sheet]
#                         for row in frame.iterrows()])
#     oldrets = flatten([[sheet for sheet in loadimages(join(indir, row[0]))
#                          if "_task-retrieval" in sheet]
#                         for row in frame.iterrows()])
#     newevnts = flatten([[join(row[1]['newdirs'], bname(sheet))
#                          for sheet in loadimages(join(indir, row[0]))
#                          if "task-Memory_events.tsv" in sheet]
#                          for row in frame.iterrows()])
#     newrets = flatten([[join(row[1]['newdirs'], bname(sheet))
#                          for sheet in loadimages(join(indir, row[0]))
#                          if  "_task-retrieval" in sheet]
#                          for row in frame.iterrows()])

#     evtmvr = tuple(zip(oldevnts, newevnts))
#     retmvr = tuple(zip(oldrets, newrets))
#     [shutil.move(item[0], item[1]) for item in evtmvr]
#     [shutil.move(item[0], item[1]) for item in retmvr]
#     savior = tuple(zip(test.sort_index().index,
#                        [item[1] for item in fullencsheets]))
#     [item[1].to_csv(join(xpu('~/cimaq_events2021'), item[0],
#                          item[0]+"_task-Memory_events.tsv"), sep='\t')
#      for item in savior]

In [48]:
display(len(sheetspersub[0]), sheetspersub[0][1][0], sheetspersub[0][1][1], sheetspersub[0][2][1], sheetspersub[0][3][1])

4

'/home/francois/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA/extracted_eprimes2021/sub-3025432/sub-3025432_task-memory_events_onset.tsv'

stimOnsetSec stimDurSec fixOnsetSec fixDurSec ImageID tNumXcond
0         11.542        3.0      14.551       5.5   False     CTL01
1         20.051        3.0      23.060       0.5   Old60     Enc01
2         23.561        3.0      26.570       1.0   Old01     Enc02
3         27.557        3.0      30.566       1.0   Old24     Enc03
4         31.569        3.0      34.578       1.5   Old68     Enc04
..           ...        ...         ...       ...     ...       ...
112      694.410        3.0     697.419       6.0   Old45     Enc75
113      703.420        3.0     706.429      10.0   False     CTL39
114      716.927        3.0     719.936       5.0   Old18     Enc76
115      724.934        3.0     727.943       1.0   Old33     Enc77
116      728.929        3.0     731.938      18.0   Old14     Enc78

[117 rows x 6 columns]

Condition  CorrectSource  Stim_RT
0         CTL              8    1.402
1         Enc              6    0.000
2         Enc              5    1.822
3         Enc              6    1.930
4         Enc              9    1.814
..        ...            ...      ...
112       Enc              6    1.974
113       CTL              9    0.836
114       Enc              5    1.201
115       Enc              5    1.249
116       Enc              8    1.150

[117 rows x 3 columns]

category                          Stim OldNumber  Recognition_ACC  \
0        OLD            animal_penguin.bmp     Old10                1   
1        OLD      sporting_bicycle_old.bmp     Old55                0   
2        OLD           fruit_lemon_new.bmp     Old69                1   
3        New            vegie_broccoli.bmp     New36                1   
4        OLD          animal_zebra_old.bmp     Old13                1   
..       ...                           ...       ...              ...   
112      New       kitchen_spatula_old.bmp     New37                0   
113      OLD        musical_violin_old.bmp     Old51                1   
114      OLD              vegie_carrot.bmp     Old78                1   
115      OLD  sporting_karate clothing.bmp     Old60                1   
116      OLD        fruit_green grapes.bmp     Old68                0   

     Recognition_RESP  Recognition_RT  Spatial_RESP  Spatial_RT  
0                   1           3.216             8         462  
1                   2           9.409             8         462  
2                   1           2.746             5        4435  
3                   2           1.633             5        4435  
4                   1           1.311             8         369  
..                ...             ...           ...         ...  
112                 1           7.013             5        1150  
113                 1           2.255             8        1748  
114                 1           1.765             8         414  
115                 1           4.964             6         518  
116                 2           1.240             6         518  

[117 rows x 8 columns]

In [ ]:
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from chardet import detect
from chardet.universaldetector import UniversalDetector as udet
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from tqdm import tqdm
from cimaq_utils import flatten
from cimaq_utils import loadimages
from cimaq_utils import get_encoding

def loadpaths(cimaq_dir='~/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA'):
#     cimaq_dir='~/../../media/francois/seagate_8tb/cimaq_03-19_data_simexp_DATA'
    cimaq_dir = xpu(cimaq_dir)
    sheetpaths = df(((sub, sorted(loadimages(join(cimaq_dir,
                                              'extracted_eprimes2021', sub))))
                  for sub in ls(join(cimaq_dir, 'extracted_eprimes2021')))).set_index(0)
    prefixes = sorted(list(dict.fromkeys([bname(str(item)).split("CIMAQ", 1)[0] 
                    for item in flatten(sheetpaths.values.tolist())
                    if "CIMAQ" in bname(item)])))
    sheetpaths[prefixes] = [val[0] for val in sheetpaths.values]
    sheetpaths = sheetpaths[prefixes]
    
    return sheetpaths, prefixes

def getid(sheet):
    nameparts = [str(item) for item
                 in os.path.splitext(bname(sheet))[0].split("_")]
    subid = "sub-"+str(nameparts[nameparts.index(\
                 (item for item in nameparts
                   if item.isnumeric()).__next__())])
    return subid

def makename(sheet):
    if prefixes[0] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_onset"+".tsv")
    if prefixes[1] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-memory_events_output"+".tsv")
    if prefixes[2] in sheet:
        return join(dname(sheet),
                    bname(dname(sheet))+"_task-Retrieval"+".tsv")
    else:
        print("python no compute: "+sheet)
        
def get_enc_onsets():
    # Names from M. St-Laurent (2019)
    EncOnsetCols = ["TrialNum", "Condition", "TrialNum_perCondi",
                    "ImageID", "Trial_part", "onsetSec", "durationSec"]
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,0]
    s_ids, yallofems = [], []
    for sheet in sheets:
        newsheet = pd.read_fwf(sheet, encoding=get_encoding(sheet)[1]['encoding'],
                               header=None, sep='\t',
                               names=EncOnsetCols).fillna(False).iloc[6:]

        stimids = newsheet[["ImageID",
                           "TrialNum_perCondi"]].drop_duplicates(subset=["ImageID",
                              "TrialNum_perCondi"]).reset_index(drop=True)
        s_ids.append((bname(sheet[0]), stimids))
        newsheets = newsheet.drop(['TrialNum_perCondi', 'Condition'], axis=1)
        tempsheet = newsheet[['TrialNum', 'Trial_part',
                              'onsetSec', 'durationSec']]

        # Extract & concatenate relevant info
        fixsheet = df([sheet[1] for sheet in tempsheet.iterrows()
                       if sheet[1]['Trial_part'] == 'Fixation'])
        tmng = tempsheet.loc[[sheet[0]
                                for sheet in tempsheet.iterrows()
                                if sheet[0] not in fixsheet.index]]
        fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
                                            "durationSec": "fixDurSec"})
        fixsheet = fixsheet.transpose().iloc[-2:].transpose().reset_index(drop=True)
        tmng = tmng.rename(columns={"onsetSec": "stimOnsetSec",
                                        "durationSec": "stimDurSec"})
        tmng = tmng.transpose().iloc[-2:].transpose().reset_index(drop=True)
        allofem = pd.concat([tmng, fixsheet, stimids], axis=1, sort=False)
        allofem = allofem.rename(columns={"TrialNum_perCondi": "tNumXcond"})
        enconsTuple = (makename(sheet), allofem)
        yallofems.append(enconsTuple)

    return sorted(yallofems)

def get_enc_outputs():
#     sheets = frame.iloc[:,1]
    encsheets = []
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,1]
    for sheet in sheets:
        encsheet = pd.read_csv(sheet, encoding=sheet[1],
                               header=0,
                               sep='\t').fillna(False).rename(
                       columns={"TrialNumber": "TrialNum",
                                "Category": "Condition"}).iloc[3:]
        encsheet = encsheet.set_index("TrialNum")
        # Drop duplicate columns with enc_onset's
        encsheet = encsheet.drop(["TrialCode", "OldNumber",
                                  "Stim_RESP"], axis=1)
        encsheet["Condition"] = encsheet["Condition"].astype('str')
        # Convert ms to s (as are all other time measures)
        encsheet["Stim_RT"] = [val/1000 for val in encsheet["Stim_RT"]]
        encsheet = encsheet.drop(["Stim_ACC"], axis=1)
        encsheet = encsheet.reset_index(drop=True)
        encoutTuple = (makename(sheet), encsheet)
        encsheets.append(encoutTuple)
    return sorted(encsheets)

def get_ret_outputs():
    frame, prefixes = loadpaths()
    sheets = frame.iloc[:,2]
    retsheets = []
    for sheet in sheets:
        # Removing last column as an Eprime error occured (St-Laurent, 2019)
        retsheet = pd.read_csv(sheet, encoding=get_encoding(sheet)[1]['encoding'], header=0,
                               sep='\t').fillna(False).iloc[:, :-1]
        retsheet["Recognition_RT"] = [float(val)/1000 for val
                                      in retsheet["Recognition_RT"]]
        retTuple = (makename(sheet), retsheet)
        retsheets.append(retTuple)
    return sorted(retsheets)
    return sheets
